# NVIDIA NIM TTS Model Deployment on Amazon SageMaker AI

## Introduction

This notebook demonstrates how to deploy the **NVIDIA NIM TTS (Magpie TTS Multilingual)** model for Text-to-Speech (TTS) tasks using Amazon SageMaker with a custom container that supports both HTTP and gRPC protocols.

### About NVIDIA NIM TTS (Magpie)

The **NVIDIA NIM TTS Magpie Multilingual** provides a production-ready text-to-speech service:

- **Architecture**: HTTP + gRPC routing to NVIDIA NIM TTS container
- **Model**: Magpie TTS Multilingual optimized for high-quality speech synthesis
- **Performance**: Low latency, high-quality audio output
- **Features**: Multiple voices, languages, zero-shot voice cloning, custom dictionaries
- **Deployment**: Ready for SageMaker real-time inference

### Key Features

1. **Dual Protocol Support**: HTTP for simple requests, gRPC for advanced features
2. **Multilingual Support**: Multiple languages and voices available
3. **Zero-Shot Voice Cloning**: Clone voices from audio prompts (gRPC)
4. **Custom Dictionaries**: Define custom pronunciations (gRPC)
5. **End-to-End Streaming**: True streaming via `InvokeEndpointWithResponseStream` API
6. **Production Ready**: Built with NVIDIA NIM for enterprise deployment

## Prerequisites and Setup

**❗ Important Notes:**
- Docker is required to pull and push container images
- You need an **NGC_API_KEY** from NVIDIA NGC ([Get one here](https://build.nvidia.com))
- ECR permissions are required for pushing Docker images to your private ECR
- NIM ECR image is currently available only in `us-east-1` region

**Supported AWS Instances (Compute Capability >= 8.0):**

| Instance Family | GPU | Examples |
|-----------------|-----|----------|
| ml.g6e.* | L40S | ml.g6e.xlarge, ml.g6e.2xlarge |
| ml.p4d.* | A100 | ml.p4d.24xlarge |
| ml.p5.* | H100 | ml.p5.48xlarge |

> ⚠️ Other GPU instances (g4dn, g5, p3) are **not supported**.

In [ ]:
# Install required packages
%pip install sagemaker>=2.246.0 boto3 soundfile --quiet

In [ ]:
# Import required packages
import boto3
import json
import sagemaker
import time
import os
from sagemaker import get_execution_role
from pathlib import Path

# Setup AWS clients and session
sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
sm_runtime = boto3.client("sagemaker-runtime")
region = sess.region_name
sts_client = sess.client('sts')
account_id = sts_client.get_caller_identity()['Account']

print(f"Region: {region}")
print(f"Account ID: {account_id}")
print(f"Role: {role}")

In [ ]:
# Define deployment arguments
public_nim_image = "public.ecr.aws/nvidia/nim:magpie-tts-multilingual-1.6.0"
nim_model = "magpie-tts-multilingual"
sm_model_name = "nim-tts-magpie-multilingual"
instance_type = "ml.g6e.xlarge"

## NIM Container Setup

Pull the NIM image from public ECR and push to your private ECR repository:

In [ ]:
# Pull NIM image from public ECR and push to private ECR

import subprocess

print(f"Public NIM Image: {public_nim_image}")
print(f"Target model name: {nim_model}")

bash_script = f"""
echo "Public NIM Image: {public_nim_image}"
docker pull {public_nim_image}

echo "Resolved account: {account_id}"
echo "Resolved region: {region}"

nim_image="{account_id}.dkr.ecr.{region}.amazonaws.com/{nim_model}"

# Ensure the repository name adheres to AWS constraints
repository_name=$(echo "{nim_model}" | tr '[:upper:]' '[:lower:]' | tr -cd '[:alnum:]._/-')

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "$repository_name" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "$repository_name" > /dev/null
    echo "✅ Created ECR repository: $repository_name"
else
    echo "✅ ECR repository already exists: $repository_name"
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin "{account_id}.dkr.ecr.{region}.amazonaws.com"

docker tag {public_nim_image} $nim_image
docker push $nim_image
echo "✅ Image pushed successfully"
echo -n $nim_image
"""

nim_image = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{nim_model}"

# Run the bash script and capture real-time output
process = subprocess.Popen(bash_script, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

while True:
    output = process.stdout.readline()
    if output == b'' and process.poll() is not None:
        break
    if output:
        print(output.decode().strip())

stderr = process.stderr.read().decode()
if stderr:
    print("Errors:", stderr)

print(f"\n🎯 Private ECR Image: {nim_image}")

In [ ]:
# Print the private ECR NIM image that will be used for SageMaker deployment
print(f"NIM Image URI: {nim_image}")

## Create SageMaker Endpoint

**Before proceeding further, please set your NGC API Key.**

In [ ]:
# SET YOUR NGC API KEY HERE
# Required for running NIM - get yours from https://build.nvidia.com
NGC_API_KEY = None  # <-- SET ME

In [ ]:
# Validate NGC API Key
assert NGC_API_KEY is not None, "NGC API KEY is not set. Please set the NGC_API_KEY variable in the previous cell."
print("✅ NGC_API_KEY is set")

In [ ]:
# Create SageMaker model with NIM container
container = {
    "Image": nim_image,
    "Environment": {
        "NGC_API_KEY": NGC_API_KEY,
        "CUDA_VISIBLE_DEVICES": "0"
    }
}

create_model_response = sm.create_model(
    ModelName=sm_model_name, 
    ExecutionRoleArn=role, 
    PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])

In [ ]:
# Create endpoint configuration
endpoint_config_name = sm_model_name

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
            "ContainerStartupHealthCheckTimeoutInSeconds": 1800,
            "InferenceAmiVersion": "al2-ami-sagemaker-inference-gpu-2"
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

In [ ]:
# Create endpoint
endpoint_name = sm_model_name

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

In [ ]:
# Wait for endpoint to be in service
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

## Inference Testing

### API Request Format

The request body follows the [NVIDIA Riva TTS SynthesizeSpeechRequest proto](https://docs.nvidia.com/nim/riva/tts/1.6.0/protos.html#nvidia-riva-tts-synthesizespeechrequest):

| Field | Type | Required | Description |
|-------|------|----------|-------------|
| `text` | string | ✅ Yes | Text to synthesize |
| `voice_name` | string | No | Voice name ([available voices](https://docs.nvidia.com/nim/riva/tts/1.10.0/support-matrix.html#available-voices)) |
| `language_code` | string | No | Language code: en-US, es-US, fr-FR, de-DE, zh-CN, vi-VN, it-IT ([docs](https://docs.nvidia.com/nim/riva/tts/1.10.0/support-matrix.html#magpie-tts-multilingual)) |
| `sample_rate_hz` | int | No | Sample rate (default: 44100) |
| `encoding` | string | No | `LINEAR_PCM` or `OGGOPUS` |
| `zero_shot_data` | object | No | `{audio_prompt, quality, transcript}` for voice cloning (gRPC only) |
| `custom_dictionary` | string | No | `"word1  pron1,word2  pron2"` with double-space separator (gRPC only) |

### Response Format

Response matches [NIM SynthesizeSpeechResponse](https://docs.nvidia.com/nim/riva/tts/1.6.0/protos.html#nvidia-riva-tts-synthesizespeechresponse):

| Field | Type | Description |
|-------|------|-------------|
| `audio` | string | Base64-encoded audio bytes |
| `meta` | object | Optional metadata from NIM |

---

### Non-Streaming Request

In [ ]:
import base64
import IPython.display as ipd

payload = {
    "text": "Hello! This is a test of NVIDIA TTS on Amazon SageMaker.",
    "language_code": "en-US",
    "sample_rate_hz": 44100,
    "encoding": "LINEAR_PCM"
}

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload)
)

result = json.loads(response["Body"].read().decode("utf8"))
print(json.dumps({k: v[:50] + "..." if k == "audio" else v for k, v in result.items()}, indent=2))

In [ ]:
# Extract audio from response and play
audio_bytes = base64.b64decode(result["audio"])
print(f"Audio size: {len(audio_bytes):,} bytes")

with open("tts_output.wav", "wb") as f:
    f.write(audio_bytes)

ipd.Audio("tts_output.wav")

### Transport Selection (Optional)

Use `CustomAttributes` header to select HTTP or gRPC transport:

| CustomAttributes | Description |
|------------------|-------------|
| `/invocations/http` | Force HTTP transport |
| `/invocations/grpc` | Force gRPC transport |
| *(not set)* | Auto-routing |

In [ ]:
payload = {
    "text": "This audio is generated using gRPC transport.",
    "language_code": "en-US",
    "sample_rate_hz": 44100
}

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    CustomAttributes="/invocations/grpc",
    Body=json.dumps(payload)
)

result = json.loads(response["Body"].read().decode("utf8"))
audio_bytes = base64.b64decode(result["audio"])
print(f"Audio size: {len(audio_bytes):,} bytes")

with open("tts_grpc.wav", "wb") as f:
    f.write(audio_bytes)

ipd.Audio("tts_grpc.wav")

### Using a Specific Voice

In [ ]:
payload = {
    "text": "This audio uses a specific voice.",
    "voice_name": "Magpie-Multilingual.EN-US.Aria",
    "language_code": "en-US",
    "sample_rate_hz": 44100
}

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload)
)

result = json.loads(response["Body"].read().decode("utf8"))
audio_bytes = base64.b64decode(result["audio"])
print(f"Audio size: {len(audio_bytes):,} bytes")

with open("tts_voice.wav", "wb") as f:
    f.write(audio_bytes)

ipd.Audio("tts_voice.wav")

### Custom Dictionary (gRPC-only)

In [ ]:
payload = {
    "text": "Welcome to NVIDIA and Amazon SageMaker integration.",
    "language_code": "en-US",
    "sample_rate_hz": 44100,
    "custom_dictionary": "NVIDIA  en-vid-ee-ah,SageMaker  sage-may-ker"
}

response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload)
)

result = json.loads(response["Body"].read().decode("utf8"))
audio_bytes = base64.b64decode(result["audio"])
print(f"Audio size: {len(audio_bytes):,} bytes")

with open("tts_custom_dict.wav", "wb") as f:
    f.write(audio_bytes)

ipd.Audio("tts_custom_dict.wav")

---

### Streaming Request

For streaming, use `invoke_endpoint_with_response_stream` with `CustomAttributes="/invocations/stream"`.

The payload is the same - streaming is enabled by the API and header, not the payload.

In [ ]:
payload = {
    "text": "Welcome to streaming text-to-speech on Amazon SageMaker. "
            "This demonstrates how audio chunks are delivered incrementally.",
    "language_code": "en-US",
    "sample_rate_hz": 44100,
    "encoding": "LINEAR_PCM"
}

response = sm_runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType="application/json",
    CustomAttributes="/invocations/stream",
    Body=json.dumps(payload)
)

In [ ]:
# Process streaming response - collect chunks
import wave
import io

chunks = []
for event in response["Body"]:
    if "PayloadPart" in event:
        chunk = event["PayloadPart"]["Bytes"]
        if chunk:
            chunks.append(chunk)
            print(f"Received chunk: {len(chunk):,} bytes")

raw_pcm = b"".join(chunks)
print(f"\nTotal PCM bytes: {len(raw_pcm):,}")

In [ ]:
# Convert PCM to WAV and play
def pcm_to_wav(pcm_data, sample_rate=44100, channels=1, bits_per_sample=16):
    buffer = io.BytesIO()
    with wave.open(buffer, "wb") as wav_file:
        wav_file.setnchannels(channels)
        wav_file.setsampwidth(bits_per_sample // 8)
        wav_file.setframerate(sample_rate)
        wav_file.writeframesraw(pcm_data)
    return buffer.getvalue()

wav_data = pcm_to_wav(raw_pcm, sample_rate=44100)
with open("tts_streaming.wav", "wb") as f:
    f.write(wav_data)

print(f"WAV file size: {len(wav_data):,} bytes")
ipd.Audio("tts_streaming.wav")

### Live Streaming Demo

Watch chunks arrive in real-time:

In [ ]:
import sys

payload = {
    "text": "This is a live streaming demonstration. Watch as each audio chunk arrives "
            "from the NVIDIA NIM TTS model in real-time.",
    "language_code": "en-US",
    "sample_rate_hz": 44100,
    "encoding": "LINEAR_PCM"
}

response = sm_runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType="application/json",
    CustomAttributes="/invocations/stream",
    Body=json.dumps(payload)
)

start_time = time.time()
chunk_count = 0
total_bytes = 0

for event in response["Body"]:
    if "PayloadPart" in event:
        chunk = event["PayloadPart"]["Bytes"]
        if chunk:
            chunk_count += 1
            total_bytes += len(chunk)
            elapsed = time.time() - start_time
            print(f"📦 Chunk {chunk_count:3d} | {len(chunk):6,} bytes | Total: {total_bytes:8,} | @{elapsed:.2f}s")
            sys.stdout.flush()

print(f"\n✅ Stream complete: {chunk_count} chunks, {total_bytes:,} bytes")

---

## Resource Cleanup

**⚠️ Cost Warning**: Delete resources when done testing.

In [ ]:
sm.delete_model(ModelName=sm_model_name)
print(f"✅ Model {sm_model_name} deleted")

sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
print(f"✅ Endpoint config {endpoint_config_name} deleted")

sm.delete_endpoint(EndpointName=endpoint_name)
print(f"✅ Endpoint {endpoint_name} deleted")

# Clean up audio files
import glob
for f in glob.glob("tts_*.wav"):
    os.remove(f)
    print(f"✅ Removed {f}")